In [ ]:
import os
import pandas as pd
import numpy as np

# People Vaccination stages

In [ ]:
target_date = "2022-12-31"

data = pd.read_csv("/content/vaccinations.csv")
specific_date_row = data[data['date'] == target_date]

In [ ]:
data.dtypes


location                                object
iso_code                                object
date                                    object
total_vaccinations                     float64
people_vaccinated                      float64
people_fully_vaccinated                float64
total_boosters                         float64
daily_vaccinations_raw                 float64
daily_vaccinations                     float64
total_vaccinations_per_hundred         float64
people_vaccinated_per_hundred          float64
people_fully_vaccinated_per_hundred    float64
total_boosters_per_hundred             float64
daily_vaccinations_per_million         float64
daily_people_vaccinated                float64
daily_people_vaccinated_per_hundred    float64
dtype: object

In [ ]:

column_sums = specific_date_row[['people_vaccinated', 'people_fully_vaccinated', 'total_boosters']].sum()
column_sum =pd.DataFrame(column_sums)

total_sum = column_sums[['people_vaccinated', 'people_fully_vaccinated', 'total_boosters']].sum()

# Calculate the percentage distribution
column_sum["percentage"] = (column_sums[['people_vaccinated', 'people_fully_vaccinated', 'total_boosters']] / total_sum) * 100
column_sum.rename_axis('vaccinations', inplace=True)
column_sum.rename(columns={0: 'total_num'}, inplace=True)
column_sum['total_num'] = column_sum['total_num'].astype(int)
column_sum.to_csv("/content/vaccinations_processed.csv")


# Deaths over month



In [ ]:
import pandas as pd
df = pd.read_csv("/content/weekly_deaths.csv")
lower_date = "2021-01-01"
upper_date = "2022-12-31"
# Use boolean indexing to filter the DataFrame for rows within the specified date range
subset_df = df[(df['date'] >= lower_date) & (df['date'] <= upper_date)]
subset_df.head()


,date,World,Afghanistan,Africa,Albania,Algeria,American Samoa,Andorra,Angola,Anguilla,...,Uruguay,Uzbekistan,Vanuatu,Vatican,Venezuela,Vietnam,Wallis and Futuna,Yemen,Zambia,Zimbabwe
364,2021-01-01,83824.0,66.0,4069.0,57.0,51.0,0.0,1.0,10.0,0.0,...,46.0,0.0,0.0,0.0,23.0,0.0,0.0,4.0,7.0,24.0
365,2021-01-02,85038.0,63.0,4206.0,56.0,40.0,0.0,1.0,9.0,0.0,...,46.0,0.0,0.0,0.0,22.0,0.0,0.0,4.0,9.0,28.0
366,2021-01-03,86299.0,63.0,4287.0,52.0,47.0,0.0,1.0,7.0,0.0,...,50.0,0.0,0.0,0.0,18.0,0.0,0.0,4.0,10.0,36.0
367,2021-01-04,87273.0,60.0,4466.0,47.0,44.0,0.0,1.0,8.0,0.0,...,54.0,0.0,0.0,0.0,19.0,0.0,0.0,3.0,11.0,31.0
368,2021-01-05,87195.0,55.0,4606.0,40.0,43.0,0.0,1.0,6.0,0.0,...,46.0,0.0,0.0,0.0,20.0,0.0,0.0,2.0,14.0,30.0


In [ ]:
subset_df['date'] = pd.to_datetime(subset_df['date'])

# Set 'date' as the index of the DataFrame (required for resampling)
subset_df.set_index('date', inplace=True)

# Resample the data on a monthly basis and sum the values for each month
resampled_df = subset_df.resample('M').sum()

print(resampled_df)


                World  Afghanistan    Africa  Albania  Algeria  \
date                                                             
2021-01-31  2999907.0       1588.0  174416.0   1322.0   1032.0   
2021-02-28  2253755.0        311.0  101386.0   2575.0    661.0   
2021-03-31  1966853.0        235.0   66690.0   3430.0    753.0   
2021-04-30  2521147.0        893.0   61519.0   1296.0    971.0   
2021-05-31  2751635.0       2005.0   63085.0    498.0   1612.0   
2021-06-30  1970705.0      12407.0   79448.0     61.0   1665.0   
2021-07-31  1838749.0      13384.0  181902.0      6.0   3081.0   
2021-08-31  2150530.0       3736.0  191359.0    202.0   7183.0   
2021-09-30  1848299.0        676.0  110243.0   1023.0   4382.0   
2021-10-31  1508797.0        448.0   54325.0   1797.0    887.0   
2021-11-30  1532264.0        295.0   35398.0   1268.0   1001.0   
2021-12-31  1529857.0        326.0   34867.0    887.0   1397.0   
2022-01-31  1639263.0        338.0   72223.0    886.0   2032.0   
2022-02-28

<ipython-input-21-ddc05e1daf77>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['date'] = pd.to_datetime(subset_df['date'])


In [ ]:
date_col_deaths = resampled_df.reset_index()
date_col_deaths['date'] = date_col_deaths['date'].dt.strftime('%Y-%m-%d')
date_col_deaths.to_csv("monthly_death_data_processed.csv")

In [ ]:
date_col_deaths.dtypes

date                  object
World                float64
Afghanistan          float64
Africa               float64
Albania              float64
                      ...   
Vietnam              float64
Wallis and Futuna    float64
Yemen                float64
Zambia               float64
Zimbabwe             float64
Length: 247, dtype: object

# Cases per month between 01-01-2021 to 31-12-2022

In [ ]:
import pandas as pd
df = pd.read_csv("/content/biweekly_cases.csv")
lower_date = "2021-01-01"
upper_date = "2022-12-31"

# Use boolean indexing to filter the DataFrame for rows within the specified date range
subset_df = df[(df['date'] >= lower_date) & (df['date'] <= upper_date)]
subset_df['date'] = pd.to_datetime(subset_df['date'])

# Set 'date' as the index of the DataFrame (required for resampling)
subset_df.set_index('date', inplace=True)
# Resample the data on a monthly basis and sum the values for each month
resampled_df = subset_df.resample('M').sum()

date_col = resampled_df.reset_index()
date_col['date'] = date_col['date'].dt.strftime('%Y-%m-%d')
date_col.to_csv("monthly_cases_data_processed.csv")


<ipython-input-24-c7ea41ccb175>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['date'] = pd.to_datetime(subset_df['date'])


In [ ]:
date_col.head()

,date,World,Afghanistan,Africa,Albania,Algeria,American Samoa,Andorra,Angola,Anguilla,...,Uruguay,Uzbekistan,Vanuatu,Vatican,Venezuela,Vietnam,Wallis and Futuna,Yemen,Zambia,Zimbabwe
0,2021-01-31,279122345.0,45450.0,11700822.0,240754.0,123457.0,0.0,25032.0,33607.0,61.0,...,305286.0,0.0,0.0,0.0,167079.0,2446.0,7.0,366.0,359646.0,253083.0
1,2021-02-28,171656704.0,13954.0,5791658.0,381950.0,89099.0,0.0,16584.0,16589.0,35.0,...,221285.0,0.0,0.0,0.0,177387.0,10852.0,63.0,956.0,406632.0,68879.0
2,2021-03-31,185882667.0,8443.0,4575259.0,322780.0,63717.0,0.0,13365.0,18280.0,66.0,...,469523.0,4492.0,28.0,0.0,233943.0,2843.0,4654.0,20390.0,180833.0,14436.0
3,2021-04-30,284089720.0,33966.0,4871447.0,117948.0,57349.0,0.0,18601.0,46783.0,374.0,...,1247579.0,85914.0,0.0,0.0,492371.0,3570.0,1509.0,33324.0,59353.0,17390.0
4,2021-05-31,311825374.0,118916.0,4029557.0,29871.0,91687.0,0.0,9422.0,103866.0,834.0,...,1240057.0,145616.0,0.0,0.0,500410.0,41754.0,53.0,8887.0,31560.0,10706.0


In [ ]:
# Specify the columns to keep in a list
columns_to_keep_deaths = ['date', 'World']

# Use indexing to select and keep the specified columns
date_col_deaths = date_col_deaths[columns_to_keep_deaths]

# Specify the columns to keep in a list
columns_to_keep = ['date', 'World']

# Use indexing to select and keep the specified columns
date_col = date_col[columns_to_keep_deaths]
date_col = date_col.rename(columns={'World': 'World_cases'})

date_col = date_col.merge(date_col_deaths)
date_col.head()

date_col.to_csv("merged_death_case_line.csv")


##line chart part 2

In [ ]:
import pandas as pd
date_col_deaths['avg'] = date_col_deaths['World'].map(lambda x: int(x/1000))
date_col_deaths.head(24)

,date,World,avg
0,2021-01-31,2999907.0,2999
1,2021-02-28,2253755.0,2253
2,2021-03-31,1966853.0,1966
3,2021-04-30,2521147.0,2521
4,2021-05-31,2751635.0,2751
5,2021-06-30,1970705.0,1970
6,2021-07-31,1838749.0,1838
7,2021-08-31,2150530.0,2150
8,2021-09-30,1848299.0,1848
9,2021-10-31,1508797.0,1508


In [ ]:
date_col_deaths.to_csv("line_chart_part2_average_monthwise.csv")

##Continent wise vaccination stacked

In [ ]:
#Finding distinct manufacturer name in the data
import pandas as pd

# Create a sample DataFrame
data = pd.read_csv("/content/vaccinations-by-manufacturer.csv")
df = pd.DataFrame(data)

# Find distinct values in 'Column1'
distinct_values = df['vaccine'].unique()

print("Distinct values in 'Column1':", distinct_values)


Distinct values in 'Column1': ['Oxford/AstraZeneca' 'Sinopharm/Beijing' 'Sputnik V' 'Pfizer/BioNTech'
 'Moderna' 'CanSino' 'Sputnik Light' 'Johnson&Johnson' 'Novavax'
 'Sanofi/GSK' 'Valneva' 'Medicago' 'Sinovac' 'Covaxin' 'SKYCovione']


Once we have the vaccinations used, we will then compute the total vaccination administered

In [ ]:
import pandas as pd
df = pd.read_csv("/content/vaccinations-by-manufacturer.csv")
lower_date = "2021-01-01"
upper_date = "2022-12-31"

# Use boolean indexing to filter the DataFrame for rows within the specified date range
subset_df = df[(df['date'] >= lower_date) & (df['date'] <= upper_date)]
subset_df['date'] = pd.to_datetime(subset_df['date'])


vacc_group_by_year = subset_df.groupby(['date', 'vaccine']).agg({'total_vaccinations': 'sum'}).unstack(1)





<ipython-input-1-b677fb5d56cc>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['date'] = pd.to_datetime(subset_df['date'])


In [ ]:

#vacc_group_by_year.columns=vacc_group_by_year.columns.droplevel()
vacc_group_by_year.head()
vacc_group_by_year.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 730 entries, 2021-01-01 to 2022-12-31
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CanSino             493 non-null    float64
 1   Covaxin             730 non-null    float64
 2   Johnson&Johnson     730 non-null    float64
 3   Medicago            4 non-null      float64
 4   Moderna             730 non-null    float64
 5   Novavax             730 non-null    float64
 6   Oxford/AstraZeneca  730 non-null    float64
 7   Pfizer/BioNTech     730 non-null    float64
 8   SKYCovione          93 non-null     float64
 9   Sanofi/GSK          730 non-null    float64
 10  Sinopharm/Beijing   730 non-null    float64
 11  Sinovac             730 non-null    float64
 12  Sputnik V           730 non-null    float64
 13  Valneva             730 non-null    float64
dtypes: float64(14)
memory usage: 85.5 KB


In [ ]:
# Set 'date' as the index of the DataFrame (required for resampling)
# Resample the data on a monthly basis and sum the values for each month

resampled_df = vacc_group_by_year.resample('M').sum()
date_vaccination = resampled_df.reset_index()
date_vaccination['date'] = date_vaccination['date'].dt.strftime('%Y-%m-%d')

date_vaccination.head()

vaccine,date,CanSino,Covaxin,Johnson&Johnson,Medicago,Moderna,Novavax,Oxford/AstraZeneca,Pfizer/BioNTech,SKYCovione,Sanofi/GSK,Sinopharm/Beijing,Sinovac,Sputnik V,Valneva
0,2021-01-31,3.0,99.0,3061.0,0.0,1.505197e+08,28.0,2.210500e+04,4.811576e+08,0.0,0.0,1954.0,2715.0,6731200.0,0.0
1,2021-02-28,4.0,272.0,7950.0,0.0,7.187832e+08,463.0,3.499502e+07,1.720373e+09,0.0,11.0,4178423.0,51477786.0,19697571.0,0.0
2,2021-03-31,5.0,509.0,47092901.0,0.0,1.817387e+09,1260.0,4.816621e+08,3.878920e+09,0.0,53.0,51222293.0,217680742.0,57159221.0,0.0
3,2021-04-30,13.0,847.0,203738438.0,0.0,3.057875e+09,2345.0,1.150393e+09,7.121222e+09,0.0,60.0,106908446.0,389185830.0,143446565.0,0.0
4,2021-05-31,8.0,1319.0,358153900.0,0.0,4.364551e+09,3241.0,1.981519e+09,1.214412e+10,0.0,62.0,176336432.0,540854323.0,235413880.0,0.0


In [ ]:
date_vaccination.to_csv("stacked_bar_chart_vaccine.csv")


In [ ]:
import pandas as pd

# Create a sample DataFrame
df = pd.read_csv("/content/vaccinations-by-manufacturer.csv")

# Convert the 'Date' column to datetime
df['date'] = pd.to_datetime(df['date'])

# Create a list of countries for which you want to find the highest date
countries_to_check = df['location'].unique()

# Initialize a dictionary to store the highest dates for each country
highest_dates = {}

# Group the DataFrame by 'Country' and find the highest date for each group
grouped = df.groupby('location')
for country in countries_to_check:
    group = grouped.get_group(country)
    highest_dates[country] = group['date'].max()



print(highest_dates)
# Print the highest dates for each country
for country, highest_date in highest_dates.items():
    print(f"The highest date for Country {country} is: {highest_date}")


{'Argentina': Timestamp('2023-10-13 00:00:00'), 'Austria': Timestamp('2023-06-30 00:00:00'), 'Belgium': Timestamp('2023-09-01 00:00:00'), 'Bulgaria': Timestamp('2023-09-01 00:00:00'), 'Canada': Timestamp('2022-12-04 00:00:00'), 'Chile': Timestamp('2022-08-30 00:00:00'), 'Croatia': Timestamp('2023-09-01 00:00:00'), 'Cyprus': Timestamp('2023-09-01 00:00:00'), 'Czechia': Timestamp('2023-09-16 00:00:00'), 'Denmark': Timestamp('2023-09-01 00:00:00'), 'Ecuador': Timestamp('2022-01-28 00:00:00'), 'Estonia': Timestamp('2023-09-01 00:00:00'), 'Finland': Timestamp('2023-08-25 00:00:00'), 'France': Timestamp('2023-07-10 00:00:00'), 'Germany': Timestamp('2023-04-07 00:00:00'), 'Hong Kong': Timestamp('2023-08-31 00:00:00'), 'Hungary': Timestamp('2023-09-01 00:00:00'), 'Iceland': Timestamp('2022-03-28 00:00:00'), 'Ireland': Timestamp('2023-09-01 00:00:00'), 'Italy': Timestamp('2023-10-12 00:00:00'), 'Japan': Timestamp('2023-05-07 00:00:00'), 'Latvia': Timestamp('2022-07-11 00:00:00'), 'Liechtenstein

In [ ]:
import pandas as pd
df = pd.read_csv("/content/vaccinations.csv")
df_class = pd.read_excel("/content/class_by_income.xlsx")
df_class = df_class.rename(columns={'Economy': 'location'})

df = df.dropna(subset=["total_vaccinations"])

date_bound = "2022-08-31"

subset_df = df[(df['date'] == date_bound)]

merged_data = pd.merge(subset_df, df_class, on="location")
merged_data.head(50)





,location,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,daily_vaccinations_per_million,daily_people_vaccinated,daily_people_vaccinated_per_hundred,Code,Region,Income group,Lending category
0,Afghanistan,AFG,2022-08-31,1.156252e+07,1.073067e+07,9.939590e+06,NaN,NaN,18980.0,28.11,26.09,24.17,NaN,461.0,12799.0,0.031,AFG,South Asia,Low income,IDA
1,Argentina,ARG,2022-08-31,1.092911e+08,4.131306e+07,3.471246e+07,30499286.0,32174.0,26972.0,240.15,90.78,76.27,67.02,593.0,1835.0,0.004,ARG,Latin America & Caribbean,Upper middle income,IBRD
2,Aruba,ABW,2022-08-31,1.731200e+05,8.968100e+04,8.343900e+04,NaN,7.0,11.0,162.62,84.24,78.38,NaN,103.0,5.0,0.005,ABW,Latin America & Caribbean,High income,NaN
3,Bahrain,BHR,2022-08-31,3.467355e+06,1.240204e+06,1.225349e+06,1001802.0,167.0,161.0,235.52,84.24,83.23,68.05,109.0,22.0,0.001,BHR,Middle East & North Africa,High income,NaN
4,Barbados,BRB,2022-08-31,3.176230e+05,1.632120e+05,1.544110e+05,NaN,18.0,16.0,112.77,57.95,54.82,NaN,57.0,7.0,0.002,BRB,Latin America & Caribbean,High income,NaN
5,Belgium,BEL,2022-08-31,2.581302e+07,9.259462e+06,9.162805e+06,7821353.0,3437.0,2350.0,221.46,79.44,78.61,67.10,202.0,57.0,0.000,BEL,Europe & Central Asia,High income,NaN
6,Bolivia,BOL,2022-08-31,1.454350e+07,7.336499e+06,6.126638e+06,2079014.0,6895.0,6216.0,118.97,60.02,50.12,17.01,509.0,1052.0,0.009,BOL,Latin America & Caribbean,Lower middle income,IBRD
7,Brazil,BRA,2022-08-31,4.703274e+08,1.865199e+08,1.716657e+08,117565662.0,211494.0,234674.0,218.44,86.63,79.73,54.60,1090.0,23112.0,0.011,BRA,Latin America & Caribbean,Upper middle income,IBRD
8,Bulgaria,BGR,2022-08-31,4.523960e+06,NaN,2.069306e+06,NaN,1307.0,1083.0,66.71,NaN,30.51,NaN,160.0,96.0,0.001,BGR,Europe & Central Asia,Upper middle income,IBRD
9,Canada,CAN,2022-08-31,8.877916e+07,3.303146e+07,3.154212e+07,24205581.0,62663.0,32965.0,230.87,85.90,82.02,62.95,857.0,9632.0,0.025,CAN,North America,High income,NaN


In [ ]:
import pandas as pd
df = pd.read_csv("/content/vaccinations.csv")

#df['date'] = pd.to_datetime(df['date'])

# Create a list of countries for which you want to find the highest date
countries_to_check = df['location'].unique()

# Initialize a dictionary to store the highest dates for each country
highest_dates = {}

# Group the DataFrame by 'Country' and find the highest date for each group
grouped = df.groupby('location')
for country in countries_to_check:
    group = grouped.get_group(country)
    highest_dates[country] = group['date'].max()

der = pd.DataFrame(highest_dates, index=[0]).T
der = der.reset_index()
der = der.rename(columns={'index': 'location', 0:"date"})


der.dtypes

location    object
date        object
dtype: object

In [ ]:
merged_df = pd.merge(df, der, on=['location', 'date'], how='inner')
merged_df.head()

,location,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,daily_vaccinations_per_million,daily_people_vaccinated,daily_people_vaccinated_per_hundred
0,Afghanistan,AFG,2023-09-26,21716120.0,18257599.0,17479727.0,2380699.0,NaN,21529.0,52.80,44.39,42.50,5.79,523.0,15640.0,0.038
1,Africa,OWID_AFR,2023-09-17,855269347.0,553200675.0,460463339.0,96808712.0,73786.0,73786.0,59.95,38.77,32.27,6.79,52.0,48289.0,0.003
2,Albania,ALB,2023-07-16,3087677.0,1349150.0,1279231.0,401767.0,NaN,111.0,108.63,47.47,45.01,14.14,39.0,12.0,0.000
3,Algeria,DZA,2022-09-04,15267442.0,7840131.0,6481186.0,575651.0,NaN,628.0,34.00,17.46,14.43,1.28,14.0,0.0,0.000
4,Andorra,AND,2023-08-06,157062.0,57912.0,53501.0,43070.0,NaN,0.0,196.71,72.53,67.01,53.94,0.0,0.0,0.000


In [ ]:
df_class = pd.read_excel("/content/class_by_income.xlsx")
df_class = df_class.rename(columns={'Economy': 'location'})

merged_income_vaccination = pd.merge(merged_df, df_class, on="location")
merged_income_vaccination.head(50)


,location,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,daily_vaccinations_per_million,daily_people_vaccinated,daily_people_vaccinated_per_hundred,Code,Region,Income group,Lending category
0,Afghanistan,AFG,2023-09-26,2.171612e+07,1.825760e+07,1.747973e+07,2380699.0,NaN,21529.0,52.80,44.39,42.50,5.79,523.0,15640.0,0.038,AFG,South Asia,Low income,IDA
1,Albania,ALB,2023-07-16,3.087677e+06,1.349150e+06,1.279231e+06,401767.0,NaN,111.0,108.63,47.47,45.01,14.14,39.0,12.0,0.000,ALB,Europe & Central Asia,Upper middle income,IBRD
2,Algeria,DZA,2022-09-04,1.526744e+07,7.840131e+06,6.481186e+06,575651.0,NaN,628.0,34.00,17.46,14.43,1.28,14.0,0.0,0.000,DZA,Middle East & North Africa,Lower middle income,IBRD
3,Andorra,AND,2023-08-06,1.570620e+05,5.791200e+04,5.350100e+04,43070.0,NaN,0.0,196.71,72.53,67.01,53.94,0.0,0.0,0.000,AND,Europe & Central Asia,High income,NaN
4,Angola,AGO,2023-09-17,2.730945e+07,1.640674e+07,9.472763e+06,2816195.0,NaN,12712.0,76.74,46.10,26.62,7.91,357.0,3554.0,0.010,AGO,Sub-Saharan Africa,Lower middle income,IBRD
5,Antigua and Barbuda,ATG,2022-09-16,1.365120e+05,6.429000e+04,6.238400e+04,9838.0,NaN,201.0,145.58,68.56,66.53,10.49,2143.0,5.0,0.005,ATG,Latin America & Caribbean,High income,IBRD
6,Argentina,ARG,2023-10-13,1.164337e+08,4.151267e+07,3.489750e+07,37192610.0,1.0,5353.0,255.84,91.22,76.68,81.72,118.0,111.0,0.000,ARG,Latin America & Caribbean,Upper middle income,IBRD
7,Armenia,ARM,2023-09-03,2.163656e+06,1.124306e+06,9.980710e+05,47379.0,NaN,53.0,77.82,40.44,35.90,1.70,19.0,18.0,0.001,ARM,Europe & Central Asia,Upper middle income,IBRD
8,Aruba,ABW,2023-10-13,1.748250e+05,9.048200e+04,NaN,NaN,NaN,1.0,164.22,84.99,NaN,NaN,9.0,0.0,0.000,ABW,Latin America & Caribbean,High income,NaN
9,Australia,AUS,2023-03-24,6.549236e+07,2.223191e+07,2.164882e+07,NaN,NaN,36214.0,250.19,84.93,82.70,NaN,1383.0,0.0,0.000,AUS,East Asia & Pacific,High income,NaN


In [ ]:

df_countries = pd.read_csv("/content/countries.csv")
df_countries = df_countries.rename(columns={'name': 'location', 'country':'iso_code'})

merge_data = pd.merge(merged_income_vaccination, df_countries, on=["location"])

In [ ]:
#since the above table has many irrelavant columns we will only focus on

In [ ]:
merge_data.head()

,location,iso_code_x,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,...,daily_vaccinations_per_million,daily_people_vaccinated,daily_people_vaccinated_per_hundred,Code,Region,Income group,Lending category,iso_code_y,latitude,longitude
0,Afghanistan,AFG,2023-09-26,21716120.0,18257599.0,17479727.0,2380699.0,NaN,21529.0,52.80,...,523.0,15640.0,0.038,AFG,South Asia,Low income,IDA,AF,33.939110,67.709953
1,Albania,ALB,2023-07-16,3087677.0,1349150.0,1279231.0,401767.0,NaN,111.0,108.63,...,39.0,12.0,0.000,ALB,Europe & Central Asia,Upper middle income,IBRD,AL,41.153332,20.168331
2,Algeria,DZA,2022-09-04,15267442.0,7840131.0,6481186.0,575651.0,NaN,628.0,34.00,...,14.0,0.0,0.000,DZA,Middle East & North Africa,Lower middle income,IBRD,DZ,28.033886,1.659626
3,Andorra,AND,2023-08-06,157062.0,57912.0,53501.0,43070.0,NaN,0.0,196.71,...,0.0,0.0,0.000,AND,Europe & Central Asia,High income,NaN,AD,42.546245,1.601554
4,Angola,AGO,2023-09-17,27309449.0,16406741.0,9472763.0,2816195.0,NaN,12712.0,76.74,...,357.0,3554.0,0.010,AGO,Sub-Saharan Africa,Lower middle income,IBRD,AO,-11.202692,17.873887


In [ ]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171 entries, 0 to 170
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   location                             171 non-null    object 
 1   iso_code_x                           171 non-null    object 
 2   date                                 171 non-null    object 
 3   total_vaccinations                   161 non-null    float64
 4   people_vaccinated                    162 non-null    float64
 5   people_fully_vaccinated              159 non-null    float64
 6   total_boosters                       145 non-null    float64
 7   daily_vaccinations_raw               31 non-null     float64
 8   daily_vaccinations                   161 non-null    float64
 9   total_vaccinations_per_hundred       161 non-null    float64
 10  people_vaccinated_per_hundred        162 non-null    float64
 11  people_fully_vaccinated_per_hund

In [ ]:
columns_to_keep = ["location", "iso_code_x", "latitude", "longitude", "people_fully_vaccinated_per_hundred", "Income group"]
final_df = merge_data[columns_to_keep]
final_df.head()

,location,iso_code_x,latitude,longitude,people_fully_vaccinated_per_hundred,Income group
0,Afghanistan,AFG,33.939110,67.709953,42.50,Low income
1,Albania,ALB,41.153332,20.168331,45.01,Upper middle income
2,Algeria,DZA,28.033886,1.659626,14.43,Lower middle income
3,Andorra,AND,42.546245,1.601554,67.01,High income
4,Angola,AGO,-11.202692,17.873887,26.62,Lower middle income


In [ ]:
final_df = final_df.dropna()
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159 entries, 0 to 170
Data columns (total 6 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   location                             159 non-null    object 
 1   iso_code_x                           159 non-null    object 
 2   latitude                             159 non-null    float64
 3   longitude                            159 non-null    float64
 4   people_fully_vaccinated_per_hundred  159 non-null    float64
 5   Income group                         159 non-null    object 
dtypes: float64(3), object(3)
memory usage: 8.7+ KB


In [ ]:
final_df.to_csv("bubble_map_data_vacc_per_hundred.csv")

In [ ]:
import pandas as pd
import numpy as np

# Sample DataFrame with NaN values
data = {'A': [1, 2, np.nan, 4],
        'B': [5, np.nan, np.nan, 8],
        'C': [9, 10, 11, 12]}

df = pd.DataFrame(data)

# Drop rows with NaN values
df = df.dropna()

# Display the DataFrame after dropping rows with NaN values
print(df)

     A    B   C
0  1.0  5.0   9
3  4.0  8.0  12


In [ ]:
import pandas as pd
df = pd.read_csv("/content/merged_death_case_stacked.csv")
df = df.reset_index()
df["date"] = pd.to_datetime(df["date"])

df.set_index('date', inplace=True)
df.to_csv("merged_death_case_stacked.csv_updated.csv")
df.head()

,index,Unnamed: 0,World_cases,World
date,,,,
2021-01-31,0,0,279122345.0,2999907.0
2021-02-28,1,1,171656704.0,2253755.0
2021-03-31,2,2,185882667.0,1966853.0
2021-04-30,3,3,284089720.0,2521147.0
2021-05-31,4,4,311825374.0,2751635.0


In [ ]:
df = df.drop('Unnamed: 0', axis=1)
df.head()

,World_cases,World
date,,
2021-01-31,279122345.0,2999907.0
2021-02-28,171656704.0,2253755.0
2021-03-31,185882667.0,1966853.0
2021-04-30,284089720.0,2521147.0
2021-05-31,311825374.0,2751635.0


In [ ]:
df.to_csv("merged_death_case_stacked.csv_updated.csv")

In [ ]:
df = pd.read_csv("/content/stacked_bar_chart_vaccine.csv")
df["date"] = pd.to_datetime(df["date"])
df.set_index('date', inplace=True)
df = df.drop("Unnamed: 0", axis=1)
df.to_csv("stacked_bar_chart_vaccine_updated.csv")

In [ ]:
import pandas as pd
df = pd.read_csv("/content/stacked_bar_chart_vaccine.csv")
columns_to_convert = ["CanSino","Covaxin","Johnson&Johnson","Medicago","Moderna","Novavax","Oxford/AstraZeneca","Pfizer/BioNTech","SKYCovione","Sanofi/GSK","Sinopharm/Beijing","Sinovac","Sputnik V","Valneva"]
df[columns_to_convert] = df[columns_to_convert].astype(int)
df = df.drop('Unnamed: 0', axis=1)

df.to_csv("stacked_bar_chart_vaccine_updated.csv")



In [2]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/owid-covid-data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348596 entries, 0 to 348595
Data columns (total 67 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   iso_code                                    348596 non-null  object 
 1   continent                                   332007 non-null  object 
 2   location                                    348596 non-null  object 
 3   date                                        348596 non-null  object 
 4   total_cases                                 310624 non-null  float64
 5   new_cases                                   339010 non-null  float64
 6   new_cases_smoothed                          337751 non-null  float64
 7   total_deaths                                289073 non-null  float64
 8   new_deaths                                  339052 non-null  float64
 9   new_deaths_smoothed                         337822 non-null  float64
 

In [3]:

column_to_keep = ["iso_code", "continent", "location", "date", "total_cases_per_million", "total_deaths_per_million"]
df = df[column_to_keep]
df = df.dropna()
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 272871 entries, 81 to 348595
Data columns (total 6 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   iso_code                  272871 non-null  object 
 1   continent                 272871 non-null  object 
 2   location                  272871 non-null  object 
 3   date                      272871 non-null  object 
 4   total_cases_per_million   272871 non-null  float64
 5   total_deaths_per_million  272871 non-null  float64
dtypes: float64(2), object(4)
memory usage: 14.6+ MB


In [4]:
df['date'] = pd.to_datetime(df['date'])

# Create a list of countries for which you want to find the highest date
countries_to_check = df['location'].unique()

# Initialize a dictionary to store the highest dates for each country
highest_dates = {}

# Group the DataFrame by 'Country' and find the highest date for each group
grouped = df.groupby('location')
for country in countries_to_check:
    group = grouped.get_group(country)
    highest_dates[country] = group['date'].max()

der = pd.DataFrame(highest_dates, index=[0]).T
der = der.reset_index()
der = der.rename(columns={'index': 'location', 0:"date"})

merged_df = pd.merge(df, der, on=['location', 'date'], how='inner')
merged_df.head()

der.dtypes

location            object
date        datetime64[ns]
dtype: object

In [5]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226 entries, 0 to 225
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   iso_code                  226 non-null    object        
 1   continent                 226 non-null    object        
 2   location                  226 non-null    object        
 3   date                      226 non-null    datetime64[ns]
 4   total_cases_per_million   226 non-null    float64       
 5   total_deaths_per_million  226 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 12.4+ KB


In [9]:

df_country_pop = pd.read_csv("/content/countries-table.csv")
column_to_keep_pop = ["country", "pop2022"]
df_country_pop = df_country_pop[column_to_keep_pop]
df_country_pop = df_country_pop.dropna()
df_country_pop = df_country_pop.rename(columns={'country':'location', 'pop2022': 'population'})
df_country_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   location    234 non-null    object 
 1   population  234 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.8+ KB


In [10]:
merged_population_stats = pd.merge(merged_df, df_country_pop, on=['location'], how='inner')
merged_population_stats.head()

,iso_code,continent,location,date,total_cases_per_million,total_deaths_per_million,population
0,AFG,Asia,Afghanistan,2023-10-12,5500.286,193.295,41128771.0
1,ALB,Europe,Albania,2023-10-12,117541.387,1267.979,2842321.0
2,DZA,Africa,Algeria,2023-10-12,6055.868,153.241,44903225.0
3,ASM,Oceania,American Samoa,2023-10-12,188712.044,767.581,44273.0
4,AND,Europe,Andorra,2023-10-12,601367.684,1991.408,79824.0


In [11]:
merged_population_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215 entries, 0 to 214
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   iso_code                  215 non-null    object        
 1   continent                 215 non-null    object        
 2   location                  215 non-null    object        
 3   date                      215 non-null    datetime64[ns]
 4   total_cases_per_million   215 non-null    float64       
 5   total_deaths_per_million  215 non-null    float64       
 6   population                215 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 13.4+ KB


In [12]:
merged_population_stats.to_csv("merged_population_stats.csv")

In [15]:
df_co = pd.read_csv("/content/countries.csv")
df_co = df_co.drop(["iso_code"], axis=1)
merged_population_stats1 = pd.merge(merged_population_stats, df_co, on=["location"], how="inner")
merged_population_stats1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 205 entries, 0 to 204
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   iso_code                  205 non-null    object        
 1   continent                 205 non-null    object        
 2   location                  205 non-null    object        
 3   date                      205 non-null    datetime64[ns]
 4   total_cases_per_million   205 non-null    float64       
 5   total_deaths_per_million  205 non-null    float64       
 6   population                205 non-null    float64       
 7   latitude                  205 non-null    float64       
 8   longitude                 205 non-null    float64       
dtypes: datetime64[ns](1), float64(5), object(3)
memory usage: 16.0+ KB


In [16]:
merged_population_stats1.to_csv("merged_population_stats.csv")

In [17]:
merged_population_stats1.head(200)

,iso_code,continent,location,date,total_cases_per_million,total_deaths_per_million,population,latitude,longitude
0,AFG,Asia,Afghanistan,2023-10-12,5500.286,193.295,41128771.0,33.939110,67.709953
1,ALB,Europe,Albania,2023-10-12,117541.387,1267.979,2842321.0,41.153332,20.168331
2,DZA,Africa,Algeria,2023-10-12,6055.868,153.241,44903225.0,28.033886,1.659626
3,ASM,Oceania,American Samoa,2023-10-12,188712.044,767.581,44273.0,-14.270972,-170.132217
4,AND,Europe,Andorra,2023-10-12,601367.684,1991.408,79824.0,42.546245,1.601554
...,...,...,...,...,...,...,...,...,...
195,USA,North America,United States,2023-10-12,305763.910,3331.912,338289857.0,37.090240,-95.712891
196,URY,South America,Uruguay,2023-10-12,303622.535,2230.340,3422794.0,-32.522779,-55.765835
197,UZB,Asia,Uzbekistan,2023-10-12,5028.439,29.341,34627652.0,41.377491,64.585262
198,VUT,Oceania,Vanuatu,2023-10-12,36784.149,42.847,326740.0,-15.376706,166.959158
